In [1]:
import stagem1.temporal_integration as sti
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

In [38]:
Um=[0.1, 0.5, 0.7]
Uw=[0, 0.1,1]
W=[sti.w2, sti.w2/2]

Var=[Um, Uw,W]
Var_Name=['velocity__um', 'velocity__uw','velocity__w']#
dim_name=['um', 'uw','w']
units_name=['m/s','m/s','s⁻¹']
selected_time=90*sti.h2s
Dt=[sti.h2s/2,sti.h2s/6,sti.h2s*3]
T=[list(np.arange(0,sti.d2s*4, Dt[0])),list(np.arange(0,sti.d2s*4, Dt[1])),list(np.arange(0,sti.d2s*4,Dt[2]))]
OT=[list(np.arange(0,sti.d2s*4, sti.h2s)),list(np.arange(0,sti.d2s*4, sti.h2s)),list(np.arange(0,sti.d2s*4-sti.h2s, sti.h2s*3))]

def sensibility_comp_ds(list_Var, 
                        list_Var_Name=['velocity__um', 'velocity__uw','velocity__w'], 
                        dim_name=['um', 'uw','w'],
                        units_name=['m/s','m/s','s⁻¹'],
                        selected_time=90*sti.h2s, 
                        Dt=[sti.h2s/2,sti.h2s/6,sti.h2s*3],
                        T=[list(np.arange(0,sti.d2s*4, 
                        Dt[0])),list(np.arange(0,sti.d2s*4, Dt[1])),list(np.arange(0,sti.d2s*4,Dt[2]))],
                        OT=[list(np.arange(0,sti.d2s*4, sti.h2s)),list(np.arange(0,sti.d2s*4, sti.h2s)),list(np.arange(0,sti.d2s*4-sti.h2s, sti.h2s*3))]):
    x_ref=sti.SetUp(time= list(np.arange(0,sti.d2s*4, sti.h2s/6)))#10 min step 
    x_ref.update_model(intmethod=sti.Runge_Kutta4)

    def batch_time(x): 
        list_ad=[]    
        for j in range (len(Dt)):
            x.update_clock(time=T[j], otime=OT[j])
            ad=x.out_ds.advancement.sel(otime=selected_time, method='nearest')
            list_ad.append((ad-ad_ref).mean('a'))
        return xr.concat(list_ad, pd.Index((Dt), name="delta_t"))

    #Velocity Variables
    for i in range (len(Var)):
        x=sti.SetUp()
        ds_b=x.batch_parameters(Var_Name[i], Var[i])
        de=ds_b.advancement.sel(otime=selected_time)# temps pris pour un zéros du cosinus
    
        x.update_model(intmethod=sti.Runge_Kutta2)
        ds_b=x.batch_parameters(Var_Name[i], Var[i])
        drk2=ds_b.advancement.sel(otime=selected_time)
    
        x.update_model(intmethod=sti.Runge_Kutta4)
        ds_b=x.batch_parameters(Var_Name[i], Var[i])
        drk4=ds_b.advancement.sel(otime=selected_time)
    

        ds_b=x_ref.batch_parameters(Var_Name[i], Var[i])
        dref=ds_b.advancement.sel(otime=selected_time)
    
        ds=xr.concat([(de-dref).mean('a'), (drk2-dref).mean('a'), (drk4-dref).mean('a')], pd.Index(["Euler", "RK2", "RK4"], name="int_method"))
        ds=ds.assign_coords({dim_name[i]:("batch", Var[i])})
        ds=ds.reset_coords(['otime', 'otime_day'], drop=True)
        ds=ds.assign_attrs(units='m')
        ds=ds.rename({'batch':dim_name[i]})
        if i==0:
            DS=ds.to_dataset(name='error_adv_'+dim_name[0], promote_attrs=True)
        else:
            DS=DS.assign({'error_adv_'+ dim_name[i]: ds})
    
    #Delta Time   
    ad_ref=x_ref.out_ds.advancement.sel(otime=selected_time, method='nearest')
    x=sti.SetUp()
    list_dm=[batch_time(x)]
    x.update_model(intmethod=sti.Runge_Kutta2)
    list_dm.append(batch_time(x))
    x.update_model(intmethod=sti.Runge_Kutta4)
    list_dm.append(batch_time(x))
    ds=xr.concat(list_dm, pd.Index(["Euler", "RK2", "RK4"], name="int_method"))
    ds=ds.reset_coords(['otime', 'otime_day'], drop=True)
    DS=DS.assign({'error_adv_delta_time': ds})
    return DS
    

In [40]:
sensibility_comp_ds(list_Var=Var)

<xarray.Dataset>
Dimensions:               (delta_t: 3, int_method: 3, um: 3, uw: 3, w: 2)
Coordinates:
  * int_method            (int_method) object 'Euler' 'RK2' 'RK4'
  * um                    (um) float64 0.1 0.5 0.7
  * uw                    (uw) float64 0.0 0.1 1.0
  * w                     (w) float64 0.0001454 7.272e-05
  * delta_t               (delta_t) float64 1.8e+03 600.0 1.08e+04
Data variables:
    error_adv_um          (int_method, um) float64 -2.123 -2.123 ... 0.006738
    error_adv_uw          (int_method, uw) float64 -2.51e-12 -2.123 ... 0.8497
    error_adv_w           (int_method, w) float64 -2.123 -0.9359 ... 0.0008654
    error_adv_delta_time  (int_method, delta_t) float64 -2.123 0.1818 ... 6.826
Attributes:
    units:    m